In [1]:
import pandas as pd
import numpy as np
import configparser
import sys
from sklearn.decomposition import PCA
config = configparser.ConfigParser()
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib)

OHCO = ['speech_id', 'para_num', 'sent_num', 'token_num']

In [2]:
LIB = pd.read_csv('LIB.csv', sep='|').set_index('speech_id')
VOCAB = pd.read_csv('VOCAB.csv', sep='|').set_index('term_str')
TFIDF = pd.read_csv('TFIDF_L2.csv').set_index(['speech_id', 'para_num'])

In [3]:
n_comps = 10
pc_cols = [f"PC{i}" for i in range(n_comps)]
pca_engine = PCA(n_components=n_comps)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF.fillna(0)), index=TFIDF.index).reset_index()
DCM['speech_id'] = DCM['speech_id'] + 1
DCM = DCM.set_index(['speech_id', 'para_num'])
DCM.columns = pc_cols
DCM = DCM.join(LIB, on='speech_id')
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = TFIDF.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)

In [4]:
data = []
for i in range(n_comps):
    for j in [0,1]:
        data.append((f"PC{i}", j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))
comp_strs = pd.DataFrame(data)
comp_strs.columns = ['pc', 'pole', 'top_terms']
comp_strs = comp_strs.set_index(['pc', 'pole'])

In [5]:
comp_strs.to_csv('components.csv')
comp_strs

top_terms
pc  pole                                                   
PC0 0     mr members president speaker audience congress...
    1     world peace nations government economic tax jo...
PC1 0     world peace nations united war freedom states ...
    1     tax health year care jobs budget federal perce...
PC2 0     thank god bless america good much americans to...
    1     audience members federal government defense bu...
PC3 0     thank audience bless god members federal membe...
    1     mr people president world americans speaker am...
PC4 0     audience members member world america usa know...
    1     mr congress president states speaker thank uni...
PC5 0     gentlemen representatives house senate health ...
    1     year years tax mr world budget percent peace s...
PC6 0     gentlemen representatives house senate years y...
    1     health states government care united federal p...
PC7 0     health care peace world nations mr security in...
    1     state union government people years federal gr...
PC8 0     united states nations gentlemen america jobs t...
    1     thank much national security war president fre...
PC9 0     new jobs economy trade energy economic america...
    1     year war peace health budget care know last un...

In [6]:
DCM.to_csv('DCM.csv')
DCM

PC0       PC1       PC2       PC3       PC4  \
speech_id para_num                                                     
1         0         0.205831  0.005653 -0.023521 -0.012175 -0.158875   
          1        -0.014996  0.110949 -0.038285  0.028145 -0.090647   
          2        -0.008564  0.009144 -0.015104  0.034474 -0.030552   
          3        -0.028436  0.116336 -0.063156  0.020987 -0.017645   
          4        -0.028593  0.049791 -0.040218  0.016126 -0.004389   
...                      ...       ...       ...       ...       ...   
114       263      -0.014451  0.018028  0.076430 -0.046979  0.054920   
          264      -0.004057 -0.028991  0.063765 -0.055304  0.041181   
          265       0.055215 -0.028619  0.144953 -0.187336  0.066130   
          266      -0.007100  0.049125  0.073843 -0.022842  0.003958   
          267       0.111857  0.085837  0.666679  0.521909 -0.142729   

                         PC5       PC6       PC7       PC8       PC9  \
speech_id para_num                                                     
1         0         0.489571  0.361118  0.098175  0.129201  0.033398   
          1         0.018797 -0.123015 -0.158420  0.079066 -0.040273   
          2         0.031981 -0.023164 -0.036983 -0.011194 -0.028648   
          3        -0.004080  0.010349  0.064467 -0.074628 -0.102150   
          4        -0.005460 -0.005663  0.056069 -0.023123 -0.011979   
...                      ...       ...       ...       ...       ...   
114       263       0.007220  0.016255 -0.016712  0.018065  0.047745   
          264       0.063881  0.072085 -0.023368  0.014713 -0.014881   
          265      -0.003707  0.007423 -0.085288  0.005060 -0.011042   
          266      -0.016273 -0.055389 -0.035998  0.274672 -0.026088   
          267      -0.083450 -0.011378  0.105573 -0.091775  0.053616   

                     president  \
speech_id para_num               
1         0         washington   
          1         washington   
          2         washington   
          3         washington   
          4         washington   
...                        ...   
114       263            biden   
          264            biden   
          265            biden   
          266            biden   
          267            biden   

                                                     source_file_path  year  \
speech_id para_num                                                            
1         0         C:/Users/patso/Documents/DS5001//data/sotu\179...  1790   
          1         C:/Users/patso/Documents/DS5001//data/sotu\179...  1790   
          2         C:/Users/patso/Documents/DS5001//data/sotu\179...  1790   
          3         C:/Users/patso/Documents/DS5001//data/sotu\179...  1790   
          4         C:/Users/patso/Documents/DS5001//data/sotu\179...  1790   
...                                                               ...   ...   
114       263       C:/Users/patso/Documents/DS5001//data/sotu\202...  2024   
          264       C:/Users/patso/Documents/DS5001//data/sotu\202...  2024   
          265       C:/Users/patso/Documents/DS5001//data/sotu\202...  2024   
          266       C:/Users/patso/Documents/DS5001//data/sotu\202...  2024   
          267       C:/Users/patso/Documents/DS5001//data/sotu\202...  2024   

                   party  
speech_id para_num        
1         0          IND  
          1          IND  
          2          IND  
          3          IND  
          4          IND  
...                  ...  
114       263        DEM  
          264        DEM  
          265        DEM  
          266        DEM  
          267        DEM  

[9399 rows x 14 columns]

In [7]:
LOADINGS.to_csv('LOADINGS.csv')
LOADINGS

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf
term_str,,,,,,,,,,,,,,,,,,,
people,-0.003365,0.000347,0.006763,-0.011926,0.004436,0.001126,-0.005065,-0.008718,0.002472,-0.002065,1924,6,0.003538,8.142906,NNS,NN,0,peopl,4049.914776
world,-0.005619,0.022832,0.000505,-0.011090,0.005764,-0.010251,0.006732,0.009931,0.000806,0.001892,1602,5,0.002946,8.407140,NN,NN,0,world,3667.352042
new,-0.002657,-0.003557,0.000197,-0.005482,0.001427,-0.005025,0.002089,-0.001748,0.004026,0.018633,1578,3,0.002902,8.428917,JJ,JJ,0,new,3624.966156
congress,0.015358,-0.004398,-0.005500,-0.003214,-0.010586,-0.000182,-0.006055,-0.003932,0.003580,-0.002739,1395,8,0.002565,8.606749,NNP,NN,0,congress,3578.575270
america,-0.002412,0.006197,0.014898,-0.007458,0.005711,-0.005199,0.003448,-0.002973,0.008622,0.006656,1412,7,0.002596,8.589274,NNP,NN,0,america,3534.140499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fairness,-0.000211,-0.000512,0.000030,-0.000120,0.000050,-0.000099,0.000147,-0.000007,0.000103,0.000002,32,8,0.000059,14.052799,NN,NN,0,fair,248.742029
active,-0.000207,0.000530,-0.000155,-0.000005,-0.000126,-0.000129,-0.000109,-0.000075,0.000046,-0.000078,33,6,0.000061,14.008405,JJ,JJ,0,activ,248.742029
transition,-0.000199,0.000111,-0.000188,-0.000078,0.000025,-0.000166,0.000178,0.000042,-0.000245,0.000150,33,10,0.000061,14.008405,NN,NN,0,transit,248.742029
